In [1]:
import requests
import pandas as pd
import sqlite3
import re


def fetch_books(language):
    url = f"https://gutendex.com/books?languages={language}"
    response = requests.get(url)
    response.raise_for_status()  
    data = response.json()
    return data['results']

In [2]:
def converse_name(author_dict):
    
    # Фамилия, Имя -> Имя Фамилия

    name_str = author_dict.get('name', '').strip()

    if not name_str:
        return ''  

    if ',' in name_str:
        # Разбиваем на Фамилию и Имя
        surname, name = name_str.split(',', 1)
        name = name.strip()
        surname = surname.strip()

        # Убираем скобки, если есть (например, "E. M. (Edward Morgan)" -> "Edward")
        match = re.search(r'\((.*?)\)', name)
        if match:
            name = match.group(1)

        return f"{name} {surname}"
    else:
        return name_str


def process_book(book):
    
    return {
        'id': book['id'],
        'title': book['title'],
        'copyright': book['copyright'],
        'media_type': book['media_type'],
        'download_count': book['download_count'],
        'authors': '; '.join([converse_name(a) for a in book['authors']]),
        'translators': '; '.join([converse_name(t) for t in book['translators']]),
        'subjects': '; '.join(book['subjects'][:4]),  
        'bookshelves': '; '.join(
            [re.sub(r'Browsing:\s*', '', shelf) for shelf in book['bookshelves'][:2]]
        ),  
        'languages': book['languages'][0] if book['languages'] else None,
        'formats': list(book['formats'].keys())[0] if book['formats'] else None
    }


books_all = []
for lang in ['en', 'es', 'fr']:
    books = fetch_books(lang)
    processed = [process_book(b) for b in books]
    books_all.extend(processed)


df = pd.DataFrame(books_all)

conn = sqlite3.connect("gutendex_books.db")
df.to_sql("books", conn, if_exists="replace", index=False)
conn.close()

print("Data successfully written to gutendex_books.db")

Data successfully written to gutendex_books.db


In [3]:
df.head()

,id,title,copyright,media_type,download_count,authors,translators,subjects,bookshelves,languages,formats
0,2701,"Moby Dick; Or, The Whale",False,Text,104393,Herman Melville,,"Adventure stories; Ahab, Captain (Fictitious c...",Best Books Ever Listings; Fiction,en,text/html
1,84,"Frankenstein; Or, The Modern Prometheus",False,Text,89216,Mary Wollstonecraft Shelley,,Frankenstein's monster (Fictitious character) ...,Culture/Civilization/Society; Fiction,en,text/html
2,7142,The History of the Peloponnesian War,False,Text,67806,Thucydides,Richard Crawley,"Greece -- History -- Peloponnesian War, 431-40...",History - European; History - General,en,text/html
3,1513,Romeo and Juliet,False,Text,66718,William Shakespeare,,Conflict of generations -- Drama; Juliet (Fict...,Fiction; Literature,en,text/html
4,2641,A Room with a View,False,Text,61156,Edward Morgan Forster,,British -- Italy -- Fiction; England -- Fictio...,Culture/Civilization/Society; Fiction,en,text/html
